In [14]:

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.layers import (
Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape,Input,Conv2D, Conv2DTranspose,LeakyReLU)

from keras.models import Sequential
from keras.optimizers import Adam
import cv2 as cv
import tensorflow as tf

In [15]:
class SqueezeLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.squeeze(inputs, axis=0)  # Squeeze the batch dimension

In [16]:
img_shape=(150,150,1)

In [18]:
def build_generator(input_shape):
 model = Sequential()
 model.add(Dense(150*150, input_dim=22500))
 model.add(Reshape((5,5, 900)))
 model.add(Conv2DTranspose(300, kernel_size=3, strides=3, padding='same'))
 model.add(BatchNormalization())
 model.add(LeakyReLU(alpha=0.01))
 model.add(Conv2DTranspose(3, kernel_size=3, strides=10, padding='same'))
 model.add(BatchNormalization())
 model.add(LeakyReLU(alpha=0.01))
 model.add(SqueezeLayer())
 return model

In [19]:
img=cv.imread('pic1.jpg')
img=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
img=cv.resize(img,(150,150))
img=img.reshape(150,150,1)
img=img.flatten()

In [33]:
generator=build_generator(img_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [35]:
img.shape
generator.summary()
# out=model.predict(img)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 22500)               │     506,272,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 5, 5, 900)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_4 (Conv2DTranspose) │ (None, 15, 15, 300)         │       2,430,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 15, 15, 300)         │           1,200 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 15, 15, 300)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_5 (Conv2DTranspose) │ (None, 150, 150, 3)         │           8,103 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 150, 150, 3)         │              12 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ squeeze_layer_2 (SqueezeLayer)       │ (150, 150, 3)               │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 508,712,115 (1.90 GB)

 Trainable params: 508,711,509 (1.90 GB)

 Non-trainable params: 606 (2.37 KB)

In [60]:
img=cv.imread('pic1.jpg')
img=cv.cvtColor(img,cv.COLOR_BGR2GRAY)


In [62]:
img=cv.resize(img,(150,150))

In [63]:
img=img.reshape(150,150,1)
img=img.flatten()

In [65]:
img=img[np.newaxis,:]


In [66]:
img.shape

(1, 22500)

In [32]:
def build_discriminator(img_shape):
 model = Sequential()
 model.add(Conv2D(32,kernel_size=3,strides=2,input_shape=(150,150,3),padding='same'))
 model.add(LeakyReLU(alpha=0.01))
 model.add(Conv2D(64,kernel_size=3,strides=2,input_shape=(150,150,3),padding='same'))
 model.add(BatchNormalization())
 model.add(LeakyReLU(alpha=0.01))
 model.add(Conv2D(128,kernel_size=3,strides=2,input_shape=(150,150,3),padding='same'))
 model.add(BatchNormalization())
 model.add(LeakyReLU(alpha=0.01))
 model.add(Flatten())
 model.add(Dense(1, activation='sigmoid'))
 return model

In [36]:
def build_gan(generator, discriminator):
 model = Sequential()
 model.add(generator)
 model.add(discriminator)
 return model
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
import tensorflow as tf
losses = []
accuracies = []
iteration_checkpoints = []
def train(iterations, batch_size, sample_interval):
 (X_train, _), (_, _) = mnist.load_data()
 X_train = X_train / 127.5 - 1.0
 X_train = np.expand_dims(X_train, axis=3)
 real = np.ones((batch_size, 1))
 fake = np.zeros((batch_size, 1))
 with tf.device('/GPU:0'):
  for iteration in range(iterations):
   idx = np.random.randint(0, X_train.shape[0], batch_size)
   imgs = X_train[idx]
   z = np.random.normal(0, 1, (batch_size, 100))
   gen_imgs = generator.predict(z,verbose=0)
   d_loss_real = discriminator.train_on_batch(imgs, real)
   d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
   d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake,)
   z = np.random.normal(0, 1, (batch_size, 100))
   gen_imgs = generator.predict(z,verbose=0)
   g_loss = gan.train_on_batch(z, real)
   if (iteration + 1) % sample_interval == 0:
    losses.append((d_loss, g_loss))
    accuracies.append(100.0 * accuracy)
    iteration_checkpoints.append(iteration + 1)
    print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
    (iteration + 1, d_loss, 100.0 * accuracy, g_loss))
    sample_images(generator)

In [120]:
 (X_train, _), (_, _) = mnist.load_data()
#  X_train = X_train / 127.5 - 1.0
#  X_train = np.expand_dims(X_train, axis=3)

In [122]:
X_train = np.expand_dims(X_train, axis=3)
X_train.shape

(60000, 28, 28, 1)

In [67]:
!pip install kaggle


In [103]:
!kaggle datasets download -d theblackmamba31/landscape-image-colorization


Dataset URL: https://www.kaggle.com/datasets/theblackmamba31/landscape-image-colorization
License(s): unknown
landscape-image-colorization.zip: Skipping, found more recently modified local copy (use --force to force download)


In [104]:
import zipfile

with zipfile.ZipFile('landscape-image-colorization.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


In [110]:
import os
image_path_y = 'dataset/landscape Images/color'
X=[]
Y=[]
for filename in os.listdir(image_path_y):
 img=cv.imread(os.path.join(image_path_y, filename))
 img=np.array(img)
 Y.append(img)

In [ ]:
def filter_images_by_shape(image_list, target_shape=(150, 150, 3)):
    filtered_images = []
    for image in image_list:
        if image.shape == target_shape:
            filtered_images.append(image)
    return filtered_images
Y=filter_images_by_shape(Y)

In [113]:
Y=np.array(Y)


In [114]:
Y.shape

(7106, 150, 150, 3)

In [115]:
for img in Y:
 img=cv.cvtColor(img,cv.COLOR_BGR2GRAY)
 img=np.array(img)
 img=img.reshape(150,150,1)
 X.append(img)

In [116]:
X=np.array(X)


In [118]:
X_normalized=X/255.0

In [119]:
Y_normalized=Y/255.0

In [81]:
import tensorflow as tf
losses = []
accuracies = []
iteration_checkpoints = []
def train(iterations, batch_size, sample_interval):
 (X_train, _), (_, _) = mnist.load_data()
 X_train = X_train / 127.5 - 1.0
 X_train = np.expand_dims(X_train, axis=3)
 real = np.ones((batch_size, 1))
 fake = np.zeros((batch_size, 1))
 with tf.device('/GPU:0'):
  for iteration in range(iterations):
   idx = np.random.randint(0, X_train.shape[0], batch_size)
   imgs = X_train[idx]
   z = np.random.normal(0, 1, (batch_size, 100))
   gen_imgs = generator.predict(z,verbose=0)
   d_loss_real = discriminator.train_on_batch(imgs, real)
   d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
   d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake,)
   z = np.random.normal(0, 1, (batch_size, 100))
   gen_imgs = generator.predict(z,verbose=0)
   g_loss = gan.train_on_batch(z, real)
   if (iteration + 1) % sample_interval == 0:
    losses.append((d_loss, g_loss))
    accuracies.append(100.0 * accuracy)
    iteration_checkpoints.append(iteration + 1)
    print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
    (iteration + 1, d_loss, 100.0 * accuracy, g_loss))
    sample_images(generator)

(150, 150, 1)